In [79]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
file_dir = '/content/drive/MyDrive/데이콘/데이콘 항공편 지연/데이콘_항공편_지연'

In [81]:
import numpy as np
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [82]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [83]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [84]:
csv_to_parquet(file_dir + '/train.csv', 'train')
csv_to_parquet(file_dir + '/test.csv', 'test')

train Done.
test Done.


In [85]:
train = pd.read_parquet('/content/train.parquet')
test = pd.read_parquet('/content/test.parquet')
sample_submission = pd.read_csv(file_dir+'/sample_submission.csv', index_col = 0)

In [86]:
after_not_delayed_test = test[test['Origin_Airport'].isin(['RIW'])]
test = test[~test['Origin_Airport'].isin(['RIW'])]

# 데이터 전처리 파이프라인

In [87]:
import itertools
from tqdm import tqdm
from datetime import datetime, timedelta
pd.set_option('mode.chained_assignment',  None)
class Processing:
    def __init__(self, x, test): # 이후 테스트 데이터도 넣는 버전 만들어야함
        self.x = x
        self.test = test
    
    def fill_airline_and_id_2(self, data):
        master_dil = '/content/drive/MyDrive/데이콘/데이콘 항공편 지연/데이콘_항공편_지연/ReleasableAircraft.2019/MASTER.txt'
        df = []
        with open(master_dil, 'r', encoding = "utf-8-sig") as file:
           df.append(file.readlines())
           
        df = list(itertools.chain(*df))  
        df = [line.split(',') for line in df]
        col = df[0]
        df = pd.DataFrame(df, columns = col)
        Airline2 = df[['N-NUMBER', 'NAME']].rename(columns={'N-NUMBER' : 'Tail_Number', 'NAME' : 'NAME2'})
        df['Tail_Number'] = ['N' + i for i in df['N-NUMBER']]
        Airline = df[['Tail_Number', 'NAME']]

        data = pd.merge(data, Airline, left_on = 'Tail_Number', right_on = 'Tail_Number', how = 'left')
        ar = pd.merge(data, Airline2, left_on = 'Tail_Number', right_on = 'Tail_Number', how = 'left')
        
        ar.loc[ar['Tail_Number'] == 'N297AK', 'NAME'] = 'ALASKA AIRLINES INC'
        ar.loc[ar['Tail_Number'] == '276NV', 'NAME'] = 'ALLEGIANT AIR LLC'
        
        ar = ar.fillna('nan')
        
        # DB-NAME을 Train Airline 기준으로 바꿈
        for base in ar.loc[~ar['Airline'].isin(['nan']), 'Airline'].unique():
            name1 = ar.loc[(ar['Airline'] == base) & ~(ar['NAME'].isin(['nan'])) & ~(ar['NAME'].isin(ar['Airline'].unique())), 'NAME'].unique()
            name2 = ar.loc[(ar['Airline'] == base) & ~(ar['NAME2'].isin(['nan'])) & ~(ar['NAME2'].isin(ar['Airline'].unique())), 'NAME2'].unique()
            ar['NAME'].replace(name1, base, inplace=True)
            ar['NAME2'].replace(name2, base, inplace=True)
        
        def apy(x):
            if x['NAME'] != 'nan':
                x['Airline'] = x['NAME']
            elif (x['NAME'] == 'nan') & (x['NAME2'] != 'nan'):
                x['Airline'] = x['NAME2']
            return x
        
        ar.loc[ar['Airline'] == 'nan'] = ar.loc[ar['Airline'] == 'nan'].apply(apy, axis=1)
        
        for n in ar[ar['Airline'] == 'nan']['Tail_Number'].unique():
            if ar.loc[ar['Tail_Number'] == n, ['Airline', 'Carrier_ID(DOT)']].shape[0] != ar.loc[(ar['Tail_Number'] == n) & ~(ar['Airline'].isin(['nan'])), ['Airline', 'Carrier_ID(DOT)']].shape[0]:
                ar.loc[ar['Tail_Number'] == n, 'Airline'] = ar.loc[(ar['Tail_Number'] == n) & ~(ar['Airline'].isin(['nan'])), 'Airline'].unique()[0]
                ar.loc[ar['Tail_Number'] == n, 'Carrier_ID(DOT)'] = ar.loc[(ar['Tail_Number'] == n) & ~(ar['Airline'].isin(['nan'])), 'Carrier_ID(DOT)'].unique()[0]
        
        ar['Airline'] = ar['Airline'].str.rstrip()
        ar['Airline'] = ar['Airline'].str.split('.').str[0]
        
        # ID 다시채움
        for airline in ar['Airline'].unique():
            id = ar.loc[(ar['Airline'] == airline) & ~(ar['Carrier_ID(DOT)'].isin(['nan'])), 'Carrier_ID(DOT)'].unique()
            ar.loc[ar['Airline'] == airline, 'Carrier_ID(DOT)'] = id[0]
        
        return ar
                
    def to_time(self, data, where = None):
        if where == None:
            print('where 넣어야함')
            return None
        
        def to_time(time_list):
            if where == 'first':
                Time = pd.Series(time_list).astype(str).str.zfill(4)
                Time = Time.replace('2400','0000')
                return [datetime.strptime(i, '%H%M').strftime("%H:%M") if i != '0nan' else np.NaN for i in Time]
            
            elif where == 'twice':
                Time = pd.Series(time_list)
                Time = Time.replace('2400','0000')
                Time = Time.replace('9999','1557')
                return [datetime.strptime(i, '%H%M').strftime("%H:%M") for i in Time]  
        
        def time_gb(x):
            if x >= 600 and x <= 659:
                return '0600-0659'
            elif x>=1400 and x<=1459:
                return '1400-1459'
            elif x>=1200 and x<=1259:
                return '1200-1259'
            elif x>=1500 and x<=1559:
                return '1500-1559'
            elif x>=1900 and x<=1959:
                return '1900-1959'
            elif x>=900 and x<=959:
                return '0900-0959'
            elif x>=1000 and x<=1059:
                return  '1000-1059'
            elif x>=2000 and x<=2059:
                return '2000-2059'
            elif x>=1300 and x<=1359:
                return '1300-1359'
            elif x>=1100 and x<=1159:
                return '1100-1159'
            elif x>=800 and x<=859:
                return '0800-0859'
            elif x>=2200 and x<=2259:
                return '2200-2259'
            elif x>=1600 and x<=1659:
                return '1600-1659'
            elif x>=1700 and x<=1759:
                return '1700-1759'
            elif x>=2100 and x<=2159:
                return '2100-2159'
            elif x>=700 and x<=759:
                return '0700-0759'
            elif x>=1800 and x<=1859:
                return '1800-1859'
            elif x>=1 and x<=559:
                return '0001-0559'
            elif x>=2300 and x<=2400:
                return '2300-2400'
            
        def distance_gb(x):
            if x < 700 :
                return 'short_route'
            elif x>=700 and x<3000:
                return 'mid_route'
            elif x>=3000:
                return 'long_route'    
        
        if where == 'first':
            time_list = [i if str(i) == 'nan' else str(int(i))  for i in data['Estimated_Departure_Time'] ]
            time_list1 = [i if str(i) == 'nan' else str(int(i))  for i in data['Estimated_Arrival_Time'] ]
            
        elif where == 'twice':
            time_list = [i for i in data['Estimated_Departure_Time']]
            time_list1 = [i for i in data['Estimated_Arrival_Time']]
        data['Estimated_Departure_Time_HH:MM'] = to_time(time_list)
        data['Estimated_Arrival_Time_HH:MM'] = to_time(time_list1)
    
        
        # 예상 비행시간 만들기 (분으로 만들기)
        data_est_time = []
        for i,j in zip(data['Estimated_Arrival_Time_HH:MM'], data['Estimated_Departure_Time_HH:MM']):
            if str(i) != 'nan' and str(j) != 'nan':
                if (datetime.strptime(str(i), "%H:%M") - datetime.strptime(str(j), "%H:%M")).total_seconds()/60 >= 0:
                    data_est_time.append((datetime.strptime(str(i), "%H:%M") - datetime.strptime(str(j), "%H:%M")).total_seconds()/60)
                else:
                    time = datetime.strptime(str(i), "%H:%M") - datetime.strptime(str(j), "%H:%M") + datetime.strptime('23:59', "%H:%M") + timedelta(minutes=1)
                    data_est_time.append(timedelta(hours=time.hour,minutes=time.minute ).total_seconds()/60)
            else:
                data_est_time.append(np.NaN)
        
        if where == 'first':
            data['Estimated_Time'] = data_est_time
            data['Dep_time_gb'] = data['Estimated_Departure_Time'].dropna().apply(time_gb)
            data['Arr_time_gb'] = data['Estimated_Arrival_Time'].dropna().apply(time_gb)
            data['route_gb'] = data['Distance'].apply(distance_gb)
            
        elif where == 'twice':
            data['Estimated_Time'] = data_est_time
            data['Dep_time_gb'] = data['Estimated_Departure_Time'].astype(int).apply(time_gb)
            data['Arr_time_gb'] = data['Estimated_Arrival_Time'].astype(int).apply(time_gb)
            data['route_gb'] = data['Distance'].apply(distance_gb)
        
        return data
        
    
    def fill_airline_and_id(self, data):
        
        for airline in data['Airline'].dropna().unique():
            id = data.loc[data['Airline'] == airline, 'Carrier_ID(DOT)'].dropna().unique()
            data.loc[data['Airline'] == airline, 'Carrier_ID(DOT)'] = id[0]
                

        for id in data['Carrier_ID(DOT)'].dropna().unique():
            airline = data.loc[data['Carrier_ID(DOT)'] == id, 'Airline'].dropna().unique()
            data.loc[data['Carrier_ID(DOT)'] == id, 'Airline'] = airline[0]
            
        return data
    
    def fill_state(self, data):
        for id in data['Origin_Airport_ID'].dropna().unique():
            # 테스트셋에 알수없는 Origin State -> dummy로 채움
            try:
                data.loc[data['Origin_Airport_ID'] == id, 'Origin_State'] = data.loc[data['Origin_Airport_ID'] == id, 'Origin_State'].dropna().unique()[0]
            except:
                data.loc[data['Origin_Airport_ID'] == id, 'Origin_State'] = 'dummy'
        
        for id in data['Destination_Airport_ID'].dropna().unique():
            try:
                data.loc[data['Destination_Airport_ID'] == id, 'Destination_State'] = data.loc[data['Destination_Airport_ID'] == id, 'Destination_State'].dropna().unique()[0]
            except: # 기록이 하나밖에 없음. Youngstown (YNG 공항)
                data.loc[data['Destination_Airport_ID'] == id, 'Destination_State'] = 'Youngstown'
        return data
        
    def fill_timedata(self, data, where=None):
        if where == 'at':
            time1 = 'Estimated_Departure_Time'
            time2 = 'Estimated_Arrival_Time'
            
        elif where == 'dt':
            time1 = 'Estimated_Arrival_Time'
            time2 = 'Estimated_Departure_Time'
            
        elif where == 'both':
            b_data = data[(data['Estimated_Arrival_Time'] != 'nan') & (data['Estimated_Departure_Time'] != 'nan')]
            b_data = b_data.groupby(['Origin_Airport', 'Destination_Airport', 'Airline'])['Estimated_Arrival_Time', 'Estimated_Departure_Time'].value_counts()
            b_data = b_data.reset_index()
            b_data = b_data.rename(columns={0:'count'})
            b_data = b_data.sort_values(['Origin_Airport', 'Destination_Airport', 'Airline', 'count'], ascending=False)
            b_data = b_data.drop_duplicates(subset=['Origin_Airport', 'Destination_Airport', 'Airline'], keep='first')

            b_data = b_data.set_index(['Origin_Airport', 'Destination_Airport', 'Airline']).drop('count', axis=1)

            b = data[(data['Estimated_Arrival_Time'] == 'nan') & (data['Estimated_Departure_Time'] == 'nan')]
            b = b.set_index(['Origin_Airport', 'Destination_Airport', 'Airline'])

            b[['Estimated_Arrival_Time', 'Estimated_Departure_Time']] = b_data[['Estimated_Arrival_Time', 'Estimated_Departure_Time']]
            
            return b.reset_index()
        
        else:
            print('Where 값을 채워주세요. at: 도착시간 보간, dt: 출발시간 보간')
            return None
            
            
        # 같은 출발/도착시간이 있을 경우 가장 많은 출발/도착시간으로 채움
        merge_data = data.groupby(['Origin_Airport', 'Destination_Airport', 'Airline'])[time2, time1, 'Estimated_Time'].value_counts()
        merge_data = merge_data.reset_index()
        merge_data = merge_data.rename(columns={0:'count'})
        merge_data = merge_data[(merge_data[time1] != 'nan') & (merge_data[time2] != 'nan')]
        merge_data = merge_data.sort_values(['Origin_Airport', 'Destination_Airport', 'Airline', 'count'], ascending=False)
        merge_data = merge_data.drop_duplicates(subset=['Origin_Airport', 'Destination_Airport', 'Airline', time1], keep='first')
        mt = data.loc[(data[time2].isin(['nan'])) & ~(data[time1].isin(['nan'])), ['Origin_Airport', 'Destination_Airport', 'Airline',time2, time1]]
        mt = mt.drop(time2, axis=1)
        mt = pd.merge(mt, merge_data, how='left', on =['Origin_Airport', 'Destination_Airport', 'Airline', time1])
        
        # 같은 출발/도착시간이 없을 경우 출도착공항, 항공사 기준 Estimated_Time이 가장 많은 시간을 채우고 출발/도착시간을 계산
        et = merge_data.groupby(['Origin_Airport', 'Destination_Airport', 'Airline'])[['Estimated_Time']].value_counts()
        et = et.reset_index()
        et = et.rename(columns={0:'count'})
        et = et.sort_values(['Origin_Airport', 'Destination_Airport', 'Airline', 'count'], ascending=False)
        et = et.drop_duplicates(subset=['Origin_Airport', 'Destination_Airport', 'Airline'], keep='first').drop(['count'], axis=1)
        mt2 = mt[mt[time2].isna()].drop(['Estimated_Time', 'count'], axis=1)
        mt2 = pd.merge(mt2, et, how='left', on =['Origin_Airport', 'Destination_Airport', 'Airline'])
        
        # 위 조건에서 Estimated_Time이 없을경우 출도착공항만 고려하여 Estimated_Time이 가장 많은 시간을 채우고 출발/도착시간을 계산
        et2 = merge_data.groupby(['Origin_Airport', 'Destination_Airport'])[['Estimated_Time']].value_counts()
        et2 = et2.reset_index()
        et2 = et2.rename(columns={0:'count'})
        et2 = et2.sort_values(['Origin_Airport', 'Destination_Airport', 'count'], ascending=False)
        et2 = et2.drop_duplicates(subset=['Origin_Airport', 'Destination_Airport'], keep='first').drop(['count'], axis=1)
        mt3= mt2[mt2['Estimated_Time'].isna()].drop(['Estimated_Time'], axis=1)
        mt3 = pd.merge(mt3, et2, how='left', on =['Origin_Airport', 'Destination_Airport'])
        
        # mt3-mt2 합침
        mt= mt.set_index(['Origin_Airport', 'Destination_Airport', 'Airline'])
        mt2= mt2.set_index(['Origin_Airport', 'Destination_Airport'])
        mt3= mt3.set_index(['Origin_Airport', 'Destination_Airport'])
        mt2.loc[mt2['Estimated_Time'].isna(), 'Estimated_Time'] = mt3['Estimated_Time']
        
        # mt2-mt 합침
        mt2= mt2.reset_index().set_index(['Origin_Airport', 'Destination_Airport', 'Airline'])
        mt.loc[mt['Estimated_Time'].isna(), 'Estimated_Time'] = mt2['Estimated_Time']
        
        # mt-data 합침 (따로 분리하여 나중에 한번에 채움. 보간한 값을 보간할 때 사용하지 않기 위함)
        mt = mt.drop('count', axis=1).reset_index().set_index(['Origin_Airport', 'Destination_Airport', 'Airline', time1])
        fill_data = data[(data[time1] != 'nan') & (data[time2] == 'nan')]
        fill_data = fill_data.set_index(['Origin_Airport', 'Destination_Airport', 'Airline', time1])
        fill_data[[time2, 'Estimated_Time']] = mt[[time2, 'Estimated_Time']]
        fill_data = fill_data.reset_index()
        
        return fill_data   
        
        
        
    
    # 시간 채우는 함수
    def fill_time(self, data, where = None):
        at_data = self.fill_timedata(data, where='at')
        dt_data = self.fill_timedata(data, where='dt')
        b_data = self.fill_timedata(data, where='both')

        data = data.set_index('ID')
        at_data = at_data.set_index('ID')
        dt_data = dt_data.set_index('ID')
        b_data = b_data.set_index('ID')
        
        data.loc[(data['Estimated_Arrival_Time'] == 'nan') & (data['Estimated_Departure_Time'] != 'nan'), ['Estimated_Arrival_Time', 'Estimated_Time']] = at_data[['Estimated_Arrival_Time', 'Estimated_Time']]
        data.loc[(data['Estimated_Departure_Time'] == 'nan')  & (data['Estimated_Arrival_Time'] != 'nan'), ['Estimated_Departure_Time', 'Estimated_Time']] = dt_data[['Estimated_Departure_Time', 'Estimated_Time']]
        data.loc[(data['Estimated_Departure_Time'] == 'nan') & (data['Estimated_Arrival_Time'] == 'nan'), ['Estimated_Departure_Time', 'Estimated_Arrival_Time', 'Estimated_Time']] = b_data[['Estimated_Departure_Time', 'Estimated_Arrival_Time', 'Estimated_Time']]
        data['Estimated_Departure_Time'] = data['Estimated_Departure_Time'].fillna(9999).astype(int).astype(str)
        data['Estimated_Departure_Time'] = data['Estimated_Departure_Time'].str.zfill(4)
        data['Estimated_Arrival_Time'] = data['Estimated_Arrival_Time'].fillna(9999).astype(int).astype(str)
        data['Estimated_Arrival_Time'] = data['Estimated_Arrival_Time'].str.zfill(4)
        
        # apply용 함수
        def arr_time(x):
            try:
                x['Estimated_Departure_Time'] = x['Estimated_Departure_Time'].replace('2400','0000')
                time = datetime.strptime(x['Estimated_Departure_Time'], "%H%M") + timedelta(minutes=x['Estimated_Time'])
                hour = str(time.hour)
                minute = str(time.minute)
                x['Estimated_Arrival_Time'] = hour+minute       
            except:
                pass
            
            return x
        
        def dep_time(x):
            try:
                x['Estimated_Arrival_Time'] = x['Estimated_Arrival_Time'].replace('2400','0000')
                time = datetime.strptime(x['Estimated_Arrival_Time'], "%H%M") + timedelta(minutes=x['Estimated_Time'])
                hour = str(time.hour)
                minute = str(time.minute)
                x['Estimated_Departure_Time'] = hour+minute       
            except:
                pass
            
            return x
        
        add_time = data.loc[(data['Estimated_Arrival_Time'] == '9999') & (data['Estimated_Departure_Time'] != '9999'), ['Estimated_Departure_Time', 'Estimated_Time', 'Estimated_Arrival_Time']]
        t = add_time.apply(arr_time, axis=1)
        t= t[~t['Estimated_Time'].isna()]
        data.loc[t.index, 'Estimated_Arrival_Time'] = t['Estimated_Arrival_Time']
        
        add_time = data.loc[(data['Estimated_Arrival_Time'] != '9999') & (data['Estimated_Departure_Time'] == '9999'), ['Estimated_Arrival_Time', 'Estimated_Time', 'Estimated_Departure_Time']]
        t = add_time.apply(dep_time, axis=1)
        t= t[~t['Estimated_Time'].isna()]
        data.loc[t.index, 'Estimated_Departure_Time'] = t['Estimated_Departure_Time']
        
        if where == 'test':
            test = data[~data['Estimated_Time'].isna()]
            test = data[data['Estimated_Time'] != 'nan']
            
            return test, data
            

        data = data[~data['Estimated_Time'].isna()]
        data = data[data['Estimated_Time'] != 'nan']

        
        return data
    
    def train_processing(self):
        data = self.fill_airline_and_id(self.x)
        data = self.fill_state(data)
        data = self.to_time(data, where='first')
        data = self.fill_airline_and_id_2(data)
        data = self.fill_time(data)
        data = self.to_time(data, where='twice')
        
        
        return data
    
    
    def test_processing(self):
        data = self.fill_airline_and_id(self.test)
        data = self.fill_state(data)
        data = self.to_time(data, where='first')
        data = self.fill_airline_and_id_2(data)
        data, origin = self.fill_time(data, where='test')
        data = self.to_time(data, where='twice')
        
    
        return data, origin
                
                
            
        

In [88]:
pc = Processing(train, test)

train = pc.train_processing()
test, origin_test = pc.test_processing()


<ipython-input-87-d06091b4e9a9>:226: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  merge_data = data.groupby(['Origin_Airport', 'Destination_Airport', 'Airline'])[time2, time1, 'Estimated_Time'].value_counts()
<ipython-input-87-d06091b4e9a9>:226: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  merge_data = data.groupby(['Origin_Airport', 'Destination_Airport', 'Airline'])[time2, time1, 'Estimated_Time'].value_counts()
<ipython-input-87-d06091b4e9a9>:205: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  b_data = b_data.groupby(['Origin_Airport', 'Destination_Airport', 'Airline'])['Estimated_Arrival_Time', 'Estimated_Departure_Time'].value_counts()
<ipython-input-87-d06091b4e9a9>:226: FutureWarning: Indexing with multiple keys (implicitly converted to a 

In [89]:
test = test[~test['Estimated_Departure_Time'].isin(['1557'])]
test = test[~test['Estimated_Arrival_Time'].isin(['1557'])]

In [90]:
def FE(x):
    x = x.drop(['NAME','NAME2', 'Carrier_Code(IATA)', 'Cancelled', 'Diverted', 'Estimated_Departure_Time', 'Estimated_Arrival_Time'], axis=1)
    x['edt_h'] = x['Estimated_Departure_Time_HH:MM'].apply(lambda x:int(x.split(':')[0]))
    x['edt_m'] = x['Estimated_Departure_Time_HH:MM'].apply(lambda x:int(x.split(':')[1]))
    x['eat_h'] = x['Estimated_Arrival_Time_HH:MM'].apply(lambda x:int(x.split(':')[0]))
    x['eat_m'] = x['Estimated_Arrival_Time_HH:MM'].apply(lambda x:int(x.split(':')[1]))

    x = x.drop(['Estimated_Departure_Time_HH:MM', 'Estimated_Arrival_Time_HH:MM'], axis=1)
    
    return x

train = FE(train)
test = FE(test)

In [91]:
from sklearn.preprocessing import LabelEncoder
train_ar = train.__deepcopy__()
test_ar = test.__deepcopy__()

label_st = train_ar.drop(columns = ['Delay'])
train_st = train_ar[(train_ar['Delay'].astype(str) != 'nan')].drop(columns = ['Delay'])
X_nan = train_ar[(train_ar['Delay'].astype(str) == 'nan')].drop(columns = ['Delay'])

qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_ID(DOT)', 'Tail_Number', 'Dep_time_gb',	'Arr_time_gb',	'route_gb']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(label_st[i])
    for label in np.unique(test[i].dropna()):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    train_ar[i] = le.transform(train_ar[i])
    train_st[i]=le.transform(train_st[i])
    X_nan[i]=le.transform(X_nan[i])
    test_ar[i]=le.transform(test_ar[i])

In [92]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import RandomForestClassifier

X = train_st
y = train_ar[(train_ar['Delay'].astype(str) != 'nan')]['Delay']

stclf = SelfTrainingClassifier(
    base_estimator = RandomForestClassifier(n_estimators = 100),
    verbose = True)

stclf.fit(X, y)

/usr/local/lib/python3.9/dist-packages/sklearn/semi_supervised/_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


SelfTrainingClassifier(base_estimator=RandomForestClassifier(), verbose=True)

In [93]:
X_nan['Delay_pred'] = stclf.predict(X_nan)

In [94]:
X_nan['Delay_pred']

ID
TRAIN_000001    Not_Delayed
TRAIN_000002    Not_Delayed
TRAIN_000003    Not_Delayed
TRAIN_000004    Not_Delayed
TRAIN_000007    Not_Delayed
                   ...     
TRAIN_999995    Not_Delayed
TRAIN_999996    Not_Delayed
TRAIN_999997    Not_Delayed
TRAIN_999998    Not_Delayed
TRAIN_999999    Not_Delayed
Name: Delay_pred, Length: 736317, dtype: object

In [95]:
train_ar.loc[train_ar['Delay'] == 'nan', 'Delay'] = X_nan['Delay_pred']

In [96]:
train_ar['Delay'].unique()

array(['Not_Delayed', 'Delayed'], dtype=object)

In [97]:
X_train = train_ar.drop('Delay', axis=1)
y_train = train_ar['Delay']
x_test = test_ar

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)


In [ ]:
y_pred = clf.predict_proba(x_test)

In [ ]:
y_pred